In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import json
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain



In [9]:

load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

In [10]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

cache_store = LocalFileStore("./cache/")

In [11]:
loader = DirectoryLoader('../data', glob="*.txt", loader_cls=TextLoader, show_progress=True)
docs = loader.load()
# pretty_print_docs(docs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2141.04it/s]


In [12]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256,
    chunk_overlap=64,
)
chunked = text_splitter.split_documents(docs)
# print(len(chunked))
# pretty_print_docs(chunked)

In [13]:
cache_store = LocalFileStore("./mxbai_cache/")

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel
model = AutoModel.from_pretrained('mixedbread-ai/mxbai-embed-large-v1', trust_remote_code=True) 

model_name = "mixedbread-ai/mxbai-embed-large-v1"
model_kwargs = {'device': 'cpu'}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/73/9c/739c7bd39907498a59f67f6e49700d5018e3916870557a0db41ec2cbe244fd34/36bfa45da00eb762ef9feebe4cff315ec779efadd08da11846bef5ba5b59b8f8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1713586106&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzU4NjEwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzczLzljLzczOWM3YmQzOTkwNzQ5OGE1OWY2N2Y2ZTQ5NzAwZDUwMThlMzkxNjg3MDU1N2EwZGI0MWVjMmNiZTI0NGZkMzQvMzZiZmE0NWRhMDBlYjc2MmVmOWZlZWJlNGNmZjMxNWVjNzc5ZWZhZGQwOGRhMTE4NDZiZWY1YmE1YjU5YjhmOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=EVguQ9DNIBFCVOSto3Qzid0DhnIfwVyJxOaiuwifOYu1Sp%7EnIUxwj3f1eIUKPffIOHyKX91uMb-M4AdyiYIDQq3C1IvRfNrbRLQu2Y52D2qpxTmb%7Ee%7Efjv8eItuRC4N0-c9fidqslJOjasaWsFXlyP8k%7E%7EDzimF3ifYxptS5nQuKp0Q7rOYfgRVkjjb%7EWS9ri2LUl0tJbCoSOpvHJFVWPAj9teJvgTp

In [22]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='mixedbread-ai/mxbai-embed-large-v1', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
emb_model = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [18]:
emb_model.model

'nomic-embed-text'

In [20]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

In [23]:
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embeddings_model, cache_store, namespace="mixedbread-ai/mxbai-embed-large-v1")

In [24]:
%%time
db = FAISS.from_documents(chunked, cached_embedder)

CPU times: user 1min 40s, sys: 1min 3s, total: 2min 44s
Wall time: 33.6 s


In [25]:
llm = Ollama(model="mistral", temperature=0)
query = "How do I defeat Dreki?"

In [26]:
retriever = db.as_retriever()
docs = retriever.get_relevant_documents(query)
pretty_print_docs(docs)

Document 1:

As you start to deplete the Dreki’s health (approximately two bars), it’ll introduce a slightly new attack into its arsenal that sees the creature leaping from afar, straight into a chomp attack. This attack is indicated by a yellow circle that will appear before it launches the attack.
Don’t be afraid to use your Resurrection Stone if you have one. There’s no checkpoint throughout this fight, so if you happen to die again before the fight is over, you’ll respawn with your stone.
Once the Dreki reaches half health, it’ll introduce another new attack to the battle, where it’ll launch an unlockable (red) attack that sees it rearing up into the air, as it begins to charge an electrical AoE attack. If you’re within the electrical circle after it finishes charging, you’ll be dealt damage and temporarily stunned, unable to move or block incoming attacks.
Higher Difficulty Tip : While the Dreki is charging up its electric attack, use that brief moment as a chance to charge either

In [27]:
%%time
print(llm.invoke(query))

 Dreki is a dragon boss found in the Elder Scrolls V: Skyrim. Defeating Dreki requires preparation, strategy, and knowledge of its strengths and weaknesses. Here are some steps to help you defeat Dreki:

1. Preparation:
   - Ensure your character is well-equipped with appropriate armor, weapons, and potions.
   - Make sure you have a good selection of spells if you're using magic.
   - Save the game before attempting to fight Dreki.

2. Location:
   - Find Dreki's lair in the northern part of Skyrim, near the Throat of the World.
   - Approach the lair cautiously, as there may be dragons guarding the entrance.

3. Strategy:
   - Use ranged attacks or spells to deal damage from a safe distance.
   - If you're using melee, try to avoid getting too close to Dreki's breath attack.
   - Focus on damaging Dreki's weak spots: its wings and head.
   - Use potions and spells to heal yourself during the fight.

4. Tactics:
   - If you have a follower, consider bringing them along for additional 

In [28]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

Document 1:

Straight out of the gate, the Dreki will spit a ball of saliva toward you – you’ll be dealt damage if you do not move in time. And while it’s arguably easier to just dodge this attack, it’s also possible to block it with your shield, or even parry it back into the creature if you’re using the Dauntless Shield, dealing damage and temporarily stunning it.
Keep in mind, though, to parry the attack, you will need to be very precise with your timing, as it’ll only work if you block the attack just before it hits you. Once the Dreki is stunned, we recommend rushing the creature and unleashing as much heavy damage as possible before the stun effect runs out. This includes using your Runic Attack. Using the Runic Attack early in a fight is generally recommended so you can get the cooldown timer going as soon as possible.
Fortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws your way can be 

In [29]:
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever
)

In [30]:
%%time
c2 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 2.04 s, sys: 2.61 s, total: 4.65 s
Wall time: 15.3 s


In [31]:
print(c2["result"])

 To defeat Dreki, you can dodge its saliva attack, block it with your shield, or parry it back to stun the creature and deal damage. The timing for parrying is crucial. Once stunned, rush in and unleash heavy damage as quickly as possible before the stun effect wears off. You can also charge up your Leviathan Axe or Blades of Chaos during its electric attack for extra damage. Avoid standing on electrified spots on the ground. Dreki does not have any unblockable attacks until it reaches half health, but blocking every attack is less efficient for dealing damage. If you need health, look beside crates found along the outskirts of the area.


In [32]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=Cohere(temperature=0), retriever=compression_retriever, chain_type="refine"
)
c3 = chain({"query": query})

/Users/jaxxsulav/Projects/LLM/Andromeda1/LangChain/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.cohere.Cohere` was deprecated in langchain-community 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


CPU times: user 1.64 s, sys: 2.11 s, total: 3.75 s
Wall time: 14.3 s


In [33]:
print(c3["result"])

 To defeat a Dreki you need to dodge, block, and parry its attacks efficiently and rush the creature to inflict as much damage as possible. You might want to consider using your Runic Attack early in the fight to start the cooldown timer as soon soon as possible. Every attack from Dreki can be blocked until it reaches half health, however, it is not efficient to do so as it makes dealing damage challenging. 

During the fight, you want to make sure you take advantage of opportunities to deal extra damage. While the Dreki is charging up its electric attack, use that brief moment to charge up your weapon. It is important to note that staying clear of the electrified spots on the ground is essential to avoid taking damage. Given the new context, it is also important to note that you can retrieve health packs located beside the crates along the outskirts of the area. 


In [34]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", 
)
c4 = chain({"query": query})

CPU times: user 1.68 s, sys: 1.42 s, total: 3.1 s
Wall time: 40.2 s


In [35]:
print(c4["result"])

 Based on the new context provided, here are some additional steps to help you defeat Dreki more effectively:
1. When Dreki charges up its electric attack, use this opportunity to charge your weapon, either the Leviathan Axe or the Blades of Chaos, for extra damage.
2. If you manage to escape the electric attack, be sure to stay clear of the electrified spots on the ground to avoid being electrified yourself. To do so, mash the circle button to free yourself from the electricity's hold.
3. Every attack from Dreki can be blocked, but it may not be very efficient as it makes it more challenging to deal damage. Focus on dealing heavy damage during stun effects and charging your weapon during Dreki's electric attack for extra damage.
4. Avoid unblockable attacks until Dreki reaches half health.
5. Use precise timing to parry the saliva attack if attempting to do so.
6. Stay clear of electrified spots on the ground to avoid being electrified yourself.
7. If you need health, try looking besi

In [52]:
%%time
chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=compression_retriever, chain_type="refine", return_source_documents=True
)
c5 = chain({"query": query})

CPU times: user 3.25 s, sys: 3.58 s, total: 6.84 s
Wall time: 38.4 s


In [53]:
print(c5['source_documents'])

[Document(page_content='Straight out of the gate, the Dreki will spit a ball of saliva toward you – you’ll be dealt damage if you do not move in time. And while it’s arguably easier to just dodge this attack, it’s also possible to block it with your shield, or even parry it back into the creature if you’re using the Dauntless Shield, dealing damage and temporarily stunning it.\nKeep in mind, though, to parry the attack, you will need to be very precise with your timing, as it’ll only work if you block the attack just before it hits you. Once the Dreki is stunned, we recommend rushing the creature and unleashing as much heavy damage as possible before the stun effect runs out. This includes using your Runic Attack. Using the Runic Attack early in a fight is generally recommended so you can get the cooldown timer going as soon as possible.\nFortunately, the Dreki does not have any unblockable attacks until it reaches half health, so technically speaking, every attack the Dreki throws you

In [54]:
print(c5["result"])

 Based on the new context provided, here are some additional steps to help you defeat Dreki more effectively:
1. When Dreki charges up its electric attack, use this opportunity to charge your weapon, either the Leviathan Axe or the Blades of Chaos, for extra damage.
2. If you manage to escape the electric attack, be sure to stay clear of the electrified spots on the ground to avoid being electrified yourself. To do so, mash the circle button to free yourself from the electricity's hold.
3. Every attack from Dreki can be blocked, but it may not be very efficient as it makes it more challenging to deal damage. Focus on dealing heavy damage during stun effects and charging your weapon during Dreki's electric attack for extra damage.
4. Avoid unblockable attacks until Dreki reaches half health.
5. Use precise timing to parry the saliva attack if attempting to do so.
6. Stay clear of electrified spots on the ground to avoid being electrified yourself.
7. If you need health, try looking besi